In [ ]:
import time
from pyspark.sql.functions import *

In [ ]:
# Config
file = "data/p2p-Gnutella05_10000.txt"
max_iter = 100
num_partition = 12
debug = False

In [ ]:
# Load the data
textFile = sc.textFile(file)
print("All lines:", textFile.count())
dataFile = textFile.filter(lambda l: len(l) > 0 and l[0] != '#')
print("Correct lines:", dataFile.count())

In [ ]:
# Create edges RDD
def to_int_tuple(line, delim='\t'):
    strings = line.split(delim)[:2]
    return (int(strings[0]), int(strings[1]))
    
edgesRDD = dataFile.map(to_int_tuple)
edgesRDD.partitionBy(num_partition)
edgesRDD.cache()

# Check that all rows have exactly 2 entries
assert edgesRDD.filter(lambda t: len(t) == 2).count() == dataFile.count()

In [ ]:
edges = edgesRDD.toDF(["fro", "to"])
edges.cache()

In [ ]:
def compose(df1, df2):
    """ Compose 2 relations represented by PairRDDs. """
    r1 = df1.alias('r1')
    r2 = df2.alias('r2')
    return r1.join(r2, col('r1.to') == col('r2.fro')).drop(col('r1.to')).drop(col('r2.fro'))

In [ ]:
n = sqlContext.createDataFrame([(0, 1), (1, 2), (2, 3)], ['fro', 'to'])
r1 = n.alias('r1')
r2 = edges.alias('r2')
r1.join(r2, 'fro').show()
compose(r1, r2).show()

In [ ]:
print("############# RDD: method 1 (single steps) ###############")
new_paths = edges
all_paths = edges

start = time.time()
true_start = start

print("NUM e:", edges.rdd.getNumPartitions())
print("NUM n:", new_paths.rdd.getNumPartitions())

# invariant:
###  - all_paths and new_paths are on 'num_partitions' partitions

last_count = all_paths.count()

for i in range(1, max_iter):
    print("________________________________")
    print("Iteration #%d:" % (i,))
    new_paths = compose(all_paths, edges)
    # Leave only really new paths
    all_paths = all_paths.union(new_paths).distinct()
    
    count = all_paths.count()
    diff_count = count - last_count
    last_count = count
    print("Number of new paths: %d\n" % (diff_count,))
    
    if debug:
        print(new_paths.take(1000), '\n')
        
    end = time.time()
    print("Iteration time: %f s." % (end - start,))
    start = end
    
    # Finish, when no more paths added
    if diff_count == 0:
        print("No new paths, finishing...")
        break

print("\n\n________________________________")
print("Total paths found: %d" % (count,))
print("Number of iterations: #%d" % (i,))

if debug:
    print()
    print(all_paths.take(1000), '\n')

true_end = time.time()
method1_time = true_end - true_start
print("\nCollecting time: %f s." % (true_end - start,))
print("Total time elapsed: %f s." % (method1_time,))
print("________________________________\n\n")

In [ ]:
print("############# RDD: method 2 (paths combining) ###############")
new_paths = edges
all_paths = edges

start = time.time()
true_start = start

# invariant:
###  - all_paths and new_paths are on 'num_partitions' partitions

last_count = all_paths.count()

for i in range(1, max_iter):
    print("________________________________")
    print("Iteration #%d:" % (i,))
    new_paths = compose(all_paths, all_paths)
    # Leave only really new paths
    all_paths = all_paths.union(new_paths).distinct()
    
    count = all_paths.count()
    diff_count = count - last_count
    last_count = count
    print("Number of new paths: %d\n" % (diff_count,))
    
    if debug:
        print(new_paths.take(1000), '\n')
        
    end = time.time()
    print("Iteration time: %f s." % (end - start,))
    start = end
    
    # Finish, when no more paths added
    if diff_count == 0:
        print("No new paths, finishing...")
        break

print("\n\n________________________________")
print("Total paths found: %d" % (all_paths.count(),))
print("Number of iterations: #%d" % (i,))

if debug:
    print()
    print(all_paths.take(1000), '\n')

true_end = time.time()
method2_time = true_end - true_start
print("\nCollecting time: %f s." % (true_end - start,))
print("Total time elapsed: %f s." % (method2_time,))
print("________________________________\n\n")

In [ ]:
print("############# RDD: method 3 (single steps + broadcast) ###############")
new_paths = edges
all_paths = edges

broadcast(edges)

start = time.time()
true_start = start

print("NUM e:", edges.rdd.getNumPartitions())
print("NUM n:", new_paths.rdd.getNumPartitions())

# invariant:
###  - all_paths and new_paths are on 'num_partitions' partitions

last_count = all_paths.count()

for i in range(1, max_iter):
    print("________________________________")
    print("Iteration #%d:" % (i,))
    new_paths = compose(all_paths, edges)
    # Leave only really new paths
    print("NUM n:", new_paths.rdd.getNumPartitions())
    all_paths = all_paths.union(new_paths).distinct()
#     all_paths.cache()
    print("NUM a:", all_paths.rdd.getNumPartitions())
    
    count = all_paths.count()
    diff_count = count - last_count
    last_count = count
    
    print("Number of new paths: %d\n" % (diff_count,))
    
    # Add new paths to all paths
#     print("NUM a:", all_paths.rdd.getNumPartitions())
#     all_paths = all_paths.unionAll(new_paths).coalesce(num_partition)
#     all_paths.cache()
    
    if debug:
        print(new_paths.take(1000), '\n')
        
    end = time.time()
    print("Iteration time: %f s." % (end - start,))
    start = end
    
    # Finish, when no more paths added
    if diff_count == 0:
        print("No new paths, finishing...")
        break

print("\n\n________________________________")
print("Total paths found: %d" % (count,))
print("Number of iterations: #%d" % (i,))

if debug:
    print()
    print(all_paths.take(1000), '\n')

true_end = time.time()
method3_time = true_end - true_start
print("\nCollecting time: %f s." % (true_end - start,))
print("Total time elapsed: %f s." % (method3_time,))
print("________________________________\n\n")

In [ ]:
print("########### Summary ############")
print("Method 1: %f s." % (method1_time,))
print("Method 2: %f s." % (method2_time,))
print("Method 2: %f s." % (method3_time,))